In [1]:
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from glob import glob
from xml.etree import ElementTree as et

In [2]:
config={
    "train_src":"./data/train",
    "valid_src":"./data/valid",
    "test_src":"./data/test",

    # Make Sure to create these folders first
    "train_dest":"./processed_data/train",
    "valid_dest":"./processed_data/valid",
    "test_dest":"./processed_data/test",

    "yaml_src":"./data.yaml"
}

In [3]:
def parse_files(dir_path:str):    
    xml_files=glob(f'{dir_path}/*.xml')
    parsed_obj_list=[]
    
    for file in xml_files:
        root=et.parse(file)
        filename=root.find('filename').text
        width=root.find('size').find('width').text
        height=root.find('size').find('height').text
    
        objects=root.findall('object')
        for object in objects:
            name=object.find('name').text
            bndbox=object.find('bndbox')
            
            xmin,xmax=bndbox.find('xmin').text,bndbox.find('xmax').text
            ymin,ymax=bndbox.find('ymin').text,bndbox.find('ymax').text
    
            parsed_obj_list.append((filename,name,*(int(e) for e in (width,height,xmin,ymin,xmax,ymax))))
    
    df=pd.DataFrame(parsed_obj_list,columns=['filename','name','width','height','xmin','ymin','xmax','ymax'])
    return df

In [4]:
train_df=parse_files(config['train_src'])
train_df.shape

(1081, 8)

In [5]:
train_df.head()

,filename,name,width,height,xmin,ymin,xmax,ymax
0,100_jpg.rf.74b204ede0834a099f56a41742941027.jpg,fox,282,178,5,31,273,179
1,101_jpg.rf.972a0fbe052e4248b14ee0ee2ddf7eae.jpg,skunk,292,172,4,23,244,153
2,102_jpg.rf.0387b37b73d94171ea62858a3daab7dc.jpg,skunk,270,174,26,43,244,166
3,102_jpg.rf.0387b37b73d94171ea62858a3daab7dc.jpg,skunk,270,174,19,6,110,101
4,103_jpg.rf.0e8e30f22d765b17cdac459d8f5342b4.jpg,skunk,148,177,19,12,129,168


In [6]:
def label_encode(name):
    encoder={'fox': 0, 'skunk': 1, 'racoon': 2, 'cow': 3, 'chicken': 4, 'person': 5, 'horse': 6, 'cat': 7, 'dog': 8, 'goat': 9}
    return encoder[name]

In [7]:
def process_df(df):
    # Label-encode class-name to cls_id
    df['cls_id']=df['name'].apply(label_encode)

    # YOLO require the center of bounding box (x_center,y_center) and height and width of bounding box (Normalized)
    # Find center of bndbox
    df['x_center']=((df['xmin']+df['xmax']) / 2) / df['width']
    df['y_center']=((df['ymin']+df['ymax']) / 2) / df['height']
    
    # Find height and width of bndbox
    df['w']=(df['xmax']-df['xmin']) / df['width']
    df['h']=(df['ymax']-df['ymin']) / df['height']

    return df

In [11]:
train_df=process_df(train_df)
train_df.head()

,filename,name,width,height,xmin,ymin,xmax,ymax,cls_id,x_center,y_center,w,h
0,100_jpg.rf.74b204ede0834a099f56a41742941027.jpg,fox,282,178,5,31,273,179,0,0.492908,0.589888,0.950355,0.831461
1,101_jpg.rf.972a0fbe052e4248b14ee0ee2ddf7eae.jpg,skunk,292,172,4,23,244,153,1,0.424658,0.511628,0.821918,0.755814
2,102_jpg.rf.0387b37b73d94171ea62858a3daab7dc.jpg,skunk,270,174,26,43,244,166,1,0.500000,0.600575,0.807407,0.706897
3,102_jpg.rf.0387b37b73d94171ea62858a3daab7dc.jpg,skunk,270,174,19,6,110,101,1,0.238889,0.307471,0.337037,0.545977
4,103_jpg.rf.0e8e30f22d765b17cdac459d8f5342b4.jpg,skunk,148,177,19,12,129,168,1,0.500000,0.508475,0.743243,0.881356


In [8]:
valid_df=parse_files(config["valid_src"])
valid_df=process_df(valid_df)

print(len(valid_df))
valid_df.head()

351


,filename,name,width,height,xmin,ymin,xmax,ymax,cls_id,x_center,y_center,w,h
0,104_jpg.rf.33db265d10eb0e656ec6b8a84835bac6.jpg,skunk,252,177,18,58,219,160,1,0.470238,0.615819,0.797619,0.576271
1,110_jpg.rf.ca06f6947e403d286173bf95bff61c8b.jpg,skunk,203,175,7,12,179,166,1,0.458128,0.508571,0.847291,0.880000
2,111_jpg.rf.9b3dfc7d56701c1f11a0fb8e9abe038c.jpg,skunk,242,180,68,53,243,162,1,0.642562,0.597222,0.723140,0.605556
3,113_jpg.rf.37efd074bf706ec7aaa336aa8178d501.jpg,skunk,157,180,1,2,158,179,1,0.506369,0.502778,1.000000,0.983333
4,118_jpg.rf.9527e2f245735b8360876e82d5ea6a43.jpg,skunk,288,162,5,1,287,147,1,0.506944,0.456790,0.979167,0.901235


In [17]:
test_df=parse_files(config["test_src"])
test_df=process_df(test_df)

print(len(test_df))
test_df.head()

157


,filename,name,width,height,xmin,ymin,xmax,ymax,cls_id,x_center,y_center,w,h
0,1000_jpg.rf.884968f09695e4a51b03989a0de3945e.jpg,person,173,146,11,18,58,94,5,0.199422,0.383562,0.271676,0.520548
1,1000_jpg.rf.884968f09695e4a51b03989a0de3945e.jpg,person,173,146,97,24,146,96,5,0.702312,0.410959,0.283237,0.493151
2,1000_jpg.rf.884968f09695e4a51b03989a0de3945e.jpg,horse,173,146,21,32,48,117,6,0.199422,0.510274,0.156069,0.582192
3,1000_jpg.rf.884968f09695e4a51b03989a0de3945e.jpg,horse,173,146,95,40,137,121,6,0.670520,0.551370,0.242775,0.554795
4,107_jpg.rf.cde52d96d00fd611d4c474f3f461531a.jpg,skunk,270,175,2,2,208,173,1,0.388889,0.500000,0.762963,0.977143


## Image files and Text files

In [18]:
grouped_obj=train_df.groupby('filename')

len(grouped_obj)

700

In [19]:
grp=grouped_obj.get_group('102_jpg.rf.0387b37b73d94171ea62858a3daab7dc.jpg')
grp

,filename,name,width,height,xmin,ymin,xmax,ymax,cls_id,x_center,y_center,w,h
2,102_jpg.rf.0387b37b73d94171ea62858a3daab7dc.jpg,skunk,270,174,26,43,244,166,1,0.500000,0.600575,0.807407,0.706897
3,102_jpg.rf.0387b37b73d94171ea62858a3daab7dc.jpg,skunk,270,174,19,6,110,101,1,0.238889,0.307471,0.337037,0.545977


In [26]:
cols=['filename','cls_id','x_center','y_center','w','h']
grp[cols].set_index('filename').to_csv('sample.txt',sep=' ',header=None,index=None)

In [21]:
from shutil import move

def process_data(srcdir,destdir,df):
    cols=['filename','cls_id','x_center','y_center','w','h']
    grouped_obj=df[cols].groupby('filename')

    images=(glob(f'{srcdir}/*jpg')) # chk what all files are present
    images=list(map(lambda fname:fname.replace('\\','/'),images))
    for grp in grouped_obj:
        grp_name=grp[0]
        img_file=f"{srcdir}/{grp_name}"

        # If file don't exist dont process it
        if img_file not in images:
            print(f'{img_file} does not exist...\n')
            continue
            
        grp_info=grp[1].set_index('filename')
        filename=os.path.splitext(grp_name)[0]

        # Create a txt file at destn for labels
        grp_info.to_csv(f"{destdir}/{filename}.txt",sep=' ',index=False,header=None)
        # move image file to destdir
        move(f'{img_file}',f'{destdir}')
        

In [22]:
process_data(config['train_src'],config['train_dest'],train_df)

./data/train/100_jpg.rf.74b204ede0834a099f56a41742941027.jpg does not exist...



In [23]:
process_data(config['valid_src'],config['valid_dest'],valid_df)

In [24]:
process_data(config['test_src'],config['test_dest'],test_df)

## Train your Yolo Model on Collab

In [25]:
# Check yolo_v5_training.ipynb to know how
# After Training .....